## 스펙트로그램과 특징추출 실습
본 페이지에서는 음성파일의 스펙트로그램과 멜스펙트로그램을 그려보는 실습을 진행합니다.

먼저 음성파일을 들어봅니다.

In [ ]:
from IPython.display import Audio
filename = "data/KsponSpeech_E00001.wav"
Audio(filename, autoplay=False)

librosa 패키지는 음성파일을 다루는데 널리 쓰이는 파이선 모듈입니다.
이 모듈을 이용해서 파일을 로드하고 스펙트로그램을 그립니다.
먼저 librosa 모듈을 설치합니다.

In [ ]:
!pip install librosa

In [ ]:
import librosa
samples, sampling_rate = librosa.load(filename, sr=None)
len(samples), sampling_rate

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
plt.rcParams["figure.figsize"]=12,5
from librosa import display
import numpy as np
plt.figure()
librosa.display.waveshow(y = samples, sr = sampling_rate, color='blue')
plt.show()

먼저 스펙트로그램을 그려봅니다. 변수들을 바꾸어가면서 어떤 변화가 있는지 관찰해봅니다.

In [ ]:
hop_length = 160 
n_fft = 512
win_length=320
window='hann'

stft = librosa.stft(samples, n_fft=n_fft, win_length=win_length, hop_length=hop_length)
spectrogram = np.abs(stft)**2
log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

plt.figure()
librosa.display.specshow(log_spectrogram, sr=sampling_rate, hop_length=hop_length, y_axis="hz", x_axis="time")


이제 멜스펙트로그램을 그려봅니다. 변수들을 바꾸어가면서 어떤 변화가 있는지 관찰해봅니다.

In [ ]:
hop_length = 160 
n_fft = 512
win_length=320
window='hann'

n_mels = 40
fmin=0
fmax=None

S = librosa.feature.melspectrogram(
    y=samples, 
    sr=sampling_rate, 
    hop_length=hop_length, n_fft=n_fft, win_length=win_length, window=window, 
    n_mels = n_mels, fmin=fmin, fmax=fmax)
S_dB = librosa.power_to_db(S, ref=np.max)
dim_feature, len_feature = S_dB.shape
fig = plt.figure()
librosa.display.specshow(S_dB, y_axis=None, x_axis=None)
fig.gca().set_yticks(range(0, dim_feature+1, 10))
fig.gca().set_xticks(range(0, len_feature, 100))
fig.gca().set_ylabel("Mel-freq. Index")
fig.gca().set_xlabel("Frame Index")


### SpecAug

구글에서 발표한 SpecAug 알고리즘에 대해서 실습해봅니다.
ESPnet 모듈이 필요합니다. 설치에 상당한 시간이 소요됩니다.

In [ ]:
!pip install espnet

앞서 

In [ ]:
from espnet2.asr.specaug.specaug import SpecAug
import torch

specaug = SpecAug(apply_time_warp=True,
                  time_warp_window=5,
                  num_freq_mask=2,
                  apply_freq_mask=True,
                  freq_mask_width_range=(0,30),
                  apply_time_mask=True,
                  num_time_mask=2,
                  time_mask_width_range=(0,40),
                 )
print(specaug)

S_dB_in = torch.unsqueeze(torch.from_numpy(S_dB.copy().T), 0)
specaug(S_dB_in)
S_dB_out = torch.squeeze(S_dB_in, 0).numpy().T

plt.figure()
librosa.display.specshow(S_dB_out, sr=sampling_rate, hop_length=hop_length, y_axis="mel", x_axis="time")

